In [121]:
import agentpy as ap
import random
import matplotlib.pyplot as plt
import IPython
import owlready2 as owl

# onto = owl.get_ontology("file://onto.owl")
# onto.load()

class VacuumAgent(ap.Agent):

    def setup(self):
        self.agentType = 0
        self.direction = (-1, 0)
        self.actions = (self.clean, self.move_N, self.move_S, self.move_E, self.move_W, self.move_random)
        self.rules = (self.rule_1, self.rule_2, self.rule_3, self.rule_4, self.rule_5, self.rule_6)

    # Razonamiento
    def see(self,e): # environment
        self.per = e.neighbors(self, 1)
        return self.per

    def next(self):
        for act in self.actions:
            for rule in self.rules:
                if rule(act):
                    act()
                    break

    def rule_1(self, act):
        validator = [False] * 2
        for dirty in self.model.dirties:
            if dirty.pos == self.model.grid.positions[self]:  # If Vacuum in Dirt position
                validator[0] = True
        if act == self.clean:
            validator[1] = True
        return sum(validator) == 2

    def rule_2(self, act):
        validator = [False] * 2
        for dirty in self.per:
            if dirty.pos[0] == self.model.grid.positions[self][0] - 1:  # If Vacuum in Dirt position
                validator[0] = True
        if act == self.move_N:
            validator[1] = True
        return sum(validator) == 2

    def rule_3(self, act):
        validator = [False] * 2
        for dirty in self.per:
            if dirty.pos[0] == self.model.grid.positions[self][0] + 1:  # If Vacuum in Dirt position
                validator[0] = True
        if act == self.move_S:
            validator[1] = True
        return sum(validator) == 2

    def rule_4(self, act):
        validator = [False] * 2
        for dirty in self.per:
            if dirty.pos[1] == self.model.grid.positions[self][1] + 1:  # If Vacuum in Dirt position
                validator[0] = True
        if act == self.move_E:
            validator[1] = True
        return sum(validator) == 2

    def rule_5(self, act):
        validator = [False] * 2
        for dirty in self.per:
            if dirty.pos[1] == self.model.grid.positions[self][1] - 1:  # If Vacuum in Dirt position
                validator[0] = True
        if act == self.move_W:
            validator[1] = True
        return sum(validator) == 2

    def rule_6(self, act):
        validator = [False] * 2
        if len(self.per) == 0:
            validator[0] = True
        if act == self.move_random:
            validator[1] = True
        return sum(validator) == 2

    def step(self):
        self.see(self.model.grid)
        self.next()

    # Acciones
    def forward(self):
        self.model.grid.move_by(self, self.direction)

    def turn(self):
        if self.direction == (-1, 0):
            self.direction = (0, 1)
        elif self.direction == (0, 1):
            self.direction = (1, 0)
        elif self.direction == (1, 0):
            self.direction = (0, -1)
        elif self.direction == (0, -1):
            self.direction = (-1, 0)

    def clean(self):
              # Create a list to store dirties to remove
      dirties_to_remove = []
      for dirty in self.model.dirties:
          if dirty.pos == self.model.grid.positions[self]:
              dirties_to_remove.append(dirty)  # Mark for removal

      # Remove dirties outside the loop
      for dirty in dirties_to_remove:
          self.model.grid.remove_agents(dirty)
          self.model.dirties.remove(dirty)  # Update the dirties list

    def move_N(self):
        self.direction = (0, 1)
        self.forward()

    def move_S(self):
        self.direction = (0, -1)
        self.forward()

    def move_E(self):
        self.direction = (1, 0)
        self.forward()

    def move_W(self):
        self.direction = (-1, 0)
        self.forward()

    def move_random(self):
        for _ in range(random.randint(0, 4)):
            self.turn()
        self.forward()


class DirtAgent(ap.Agent):
    def setup(self):
        self.agentType = 1
        self.first_step = True
        self.pos = (0, 0)
    def step(self):
        if self.first_step:
            self.pos = self.model.grid.positions[self]
            self.first_step = False


class VacuumModel(ap.Model):
    def setup(self):
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.vacuums = ap.AgentList(self, self.p.vacuums, VacuumAgent)
        self.dirties = ap.AgentList(self, self.p.dirties, DirtAgent)



        self.grid.add_agents(self.vacuums, random=True, empty=True)
        self.grid.add_agents(self.dirties, random=True, empty=True)

    def step(self):
        self.vacuums.step()
        self.dirties.step()

parameters = {
    "M": 10,
    "N": 10,
    "steps": 100,
    "vacuums": 1,
    "dirties": 10,
}

# model = VacuumModel(parameters)
# results = model.run



In [95]:
def animation_plot(model,ax):
    agent_type_grid=model.grid.attr_grid("agentType")
    ap.gridplot(agent_type_grid,cmap="Accent",ax=ax)
    ax.set_title(f"Vaccum Model \n Time-step:{model.t},"f"Drity:{0}")




In [123]:
fig, ax = plt.subplots()
model = VacuumModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())